In [1]:
!git pull

Already up to date.


In [3]:
import numpy as np
import torch
from environment.Gridworld import Gridworld
from IPython.display import clear_output
import random
from matplotlib import pylab as plt
from collections import deque
from common.test import *
from environment.MarketEnv import MarketEnv
from agent.properties import *

for i in range(epochs):
    marketEnv = MarketEnv()
    state1_ = marketEnv.reset()

    state1 = torch.from_numpy(state1_).float().to(device = devid)
    status = 1
    mov = 0

    while(status == 1): 
        j+=1
        mov += 1
        qval = model(state1)
        
        if not torch.cuda.is_available():
            qval_ = qval.data.numpy()
        else:
            qval_ = qval.data.cpu().numpy()
        
        if (random.random() < epsilon):
            action_ = np.random.randint(0,4)
        else:
            action_ = np.argmax(qval_)
        
        action = action_
        print(action)
        
        # Execute action and upate state, and get reward + boolTerminal
        marketEnv.step(action)
        
        # rendered_game_boad_2 = game.board.render_np()
        state2_, reward, done, info_dic = marketEnv.step(action)
        # state2_ = game.board.render_np().reshape(1,64) + np.random.rand(1,64)/100.0
        state2 = torch.from_numpy(state2_).float()

        exp =  (state1, action_, reward, state2, done)
        replay.append(exp) #H
        state1 = state2
        
        if len(replay) > batch_size:
            minibatch = random.sample(replay, batch_size)

            # Could be replaced with pytorch gather
            state1_batch = torch.cat([s1 for (s1,a,r,s2,d) in minibatch]).view(batch_size, l1).to(device = devid)
            action_batch = torch.tensor([a for (s1,a,r,s2,d) in minibatch]).type(torch.FloatTensor).to(device = devid)
            reward_batch = torch.tensor([r for (s1,a,r,s2,d) in minibatch]).type(torch.FloatTensor).to(device = devid)
            state2_batch = torch.cat([s2 for (s1,a,r,s2,d) in minibatch]).view(batch_size, l1).to(device = devid)
            done_batch = torch.tensor([d for (s1,a,r,s2,d) in minibatch]).type(torch.FloatTensor).to(device = devid)

            # Q update
            Q1 = model(state1_batch).to(device = devid)
            with torch.no_grad():
                Q2 = model2(state2_batch).to(device = devid) #B
            
            Y = reward_batch + gamma * ((1-done_batch) * torch.max(Q2,dim=1)[0])
            X = Q1.gather(dim=1,index=action_batch.long().unsqueeze(dim=1)).squeeze()
            loss = loss_fn(X, Y.detach())
            print(i, loss.item())
            # clear_output(wait=True)
            optimizer.zero_grad()
            loss.backward()
            losses.append(loss.item())
            optimizer.step()
            
            if j % sync_freq == 0: #C
                model2.load_state_dict(model.state_dict())
        if done or mov > max_moves:
            status = 0
            mov = 0
        
losses = np.array(losses)

plt.figure(figsize=(10,7))
plt.plot(losses)
plt.xlabel("Epochs",fontsize=22)
plt.ylabel("Loss",fontsize=22)

0
0 513.9256591796875
2
0 549.933837890625
0
0 1293.608642578125
2
0 1245.2554931640625
0
0 604.03369140625
1
0 672.3389282226562
1
0 637.406005859375
2
0 1007.72314453125
0
0 1332.7237548828125
1
0 762.873779296875
0
0 773.4631958007812
0
0 753.47314453125
2
0 839.9435424804688
2
0 805.9043579101562
2
0 835.4029541015625
2
0 643.6495971679688
1
0 1048.369140625
1
0 457.4663391113281
0
0 899.265869140625
2
0 901.38818359375
3
0 659.257080078125
0
0 650.5118408203125
3
0 596.4720458984375
0
0 544.3685302734375
0
0 1510.3271484375
3
0 676.796875
0
0 675.6795043945312
2
0 471.1875915527344
1
0 726.968505859375
1
0 591.016845703125
1
0 822.8739013671875
1
0 326.73455810546875
2
0 802.9874877929688
1
0 966.1038818359375
3
0 741.1837768554688
1
0 1243.6767578125
2
0 1007.0546875
1
0 851.9765014648438
2
0 993.968505859375
2
0 985.7945556640625
3
0 654.112548828125
1
0 828.2047119140625
1
0 1223.1298828125
2
0 951.9774169921875
3
0 827.6835327148438
1
0 721.1034545898438
3
0 436.3843688964844


KeyboardInterrupt: 

In [3]:
model

Sequential(
  (0): Linear(in_features=2, out_features=150, bias=True)
  (1): ReLU()
  (2): Linear(in_features=150, out_features=100, bias=True)
  (3): ReLU()
  (4): Linear(in_features=100, out_features=101, bias=True)
)

In [4]:
minibatch

[(tensor([2500.0000,   37.6717], device='cuda:0'),
  1,
  53.0,
  tensor([2.4020e+03, 1.0000e+00]),
  False),
 (tensor([2500.0000,   49.7582], device='cuda:0'),
  0,
  0.0,
  tensor([2471.,    0.]),
  False),
 (tensor([2500.0000,   39.1841], device='cuda:0'),
  3,
  15.0,
  tensor([2489.,    3.]),
  False),
 (tensor([2500.0000,   68.1300], device='cuda:0'),
  0,
  0.0,
  tensor([2474.,    0.]),
  False),
 (tensor([2500.0000,   63.9114], device='cuda:0'),
  1,
  25.0,
  tensor([2.4590e+03, 1.0000e+00]),
  False),
 (tensor([2.5000e+03, 1.0197e+00], device='cuda:0'),
  3,
  33.0,
  tensor([2482.,    3.]),
  False),
 (tensor([2500.0000,   73.7868], device='cuda:0'),
  3,
  81.0,
  tensor([2431.,    3.]),
  False),
 (tensor([2500.0000,   45.9809], device='cuda:0'),
  2,
  48.0,
  tensor([2.4450e+03, 2.0000e+00]),
  False),
 (tensor([2500.0000,   70.4715], device='cuda:0'),
  0,
  0.0,
  tensor([2472.,    0.]),
  False),
 (tensor([2500.0000,   60.9973], device='cuda:0'),
  3,
  90.0,
  tenso

In [5]:
state1_ = marketEnv.reset()
state1 = torch.from_numpy(state1_).float().to(device = devid)
state1

tensor([2500.0000,   44.0127], device='cuda:0')

In [6]:
qval = model(state1)
qval

tensor([ 4.7263e+02,  5.0482e+02,  5.2627e+02,  5.4320e+02, -1.7049e+01,
        -2.8721e+02,  2.7055e+02, -1.4449e+02,  6.5888e+01, -1.6998e+02,
         1.3855e+02,  9.7636e+01,  1.4713e+02, -5.3032e+00, -2.4343e+01,
        -2.9656e+00,  1.3266e+02,  2.9187e+02,  3.2470e+02, -1.2648e+02,
         1.6125e+02,  2.8317e+01,  1.1781e+02, -3.0612e+02,  2.0932e+02,
         2.5163e+02,  1.9913e+02,  1.5835e+02, -1.6365e+02,  1.5549e+02,
        -3.4901e+02,  1.2267e+02, -1.7787e+02, -2.0185e+02, -3.1377e+02,
        -3.1752e+01, -2.1371e+02,  1.2422e+02, -1.3482e+02, -5.9435e+02,
        -2.0493e+02,  2.4408e-01,  5.8703e+01, -1.2842e+02, -5.7260e+01,
        -2.3565e+02,  1.0769e+02,  1.2792e+01, -6.3768e+01,  3.2637e+02,
        -1.2837e+02, -3.5006e+02,  1.4588e+01, -5.0801e+01,  5.2414e+01,
        -1.5324e+00, -5.6725e+01,  4.4175e+02,  4.0968e+02,  2.7317e+02,
        -2.8090e+02, -9.8235e+01, -1.9346e+01,  3.4278e+01,  6.0664e+01,
        -2.0080e+02,  2.3787e+02, -3.5602e+02,  6.5

In [7]:
qval_ = qval.data.cpu().numpy()
action_ = np.argmax(qval_)
action_

3

In [8]:
qval_

array([ 4.72629791e+02,  5.04819305e+02,  5.26273438e+02,  5.43196533e+02,
       -1.70491486e+01, -2.87211792e+02,  2.70553131e+02, -1.44491684e+02,
        6.58883667e+01, -1.69984467e+02,  1.38547211e+02,  9.76364670e+01,
        1.47126434e+02, -5.30320501e+00, -2.43429089e+01, -2.96556997e+00,
        1.32659195e+02,  2.91867401e+02,  3.24700287e+02, -1.26479279e+02,
        1.61249084e+02,  2.83165531e+01,  1.17813896e+02, -3.06119232e+02,
        2.09320114e+02,  2.51631531e+02,  1.99133682e+02,  1.58354416e+02,
       -1.63645950e+02,  1.55486267e+02, -3.49010681e+02,  1.22671738e+02,
       -1.77873123e+02, -2.01853577e+02, -3.13765503e+02, -3.17522507e+01,
       -2.13712982e+02,  1.24221939e+02, -1.34824585e+02, -5.94351868e+02,
       -2.04926987e+02,  2.44082183e-01,  5.87028618e+01, -1.28422897e+02,
       -5.72601891e+01, -2.35652206e+02,  1.07688599e+02,  1.27916727e+01,
       -6.37681122e+01,  3.26369904e+02, -1.28370132e+02, -3.50063965e+02,
        1.45880718e+01, -